<a href="https://colab.research.google.com/github/hamidriasat/Natural-Language-Processing/blob/master/Hotel_Review_Prediction/Hotel_Riviews_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-pretrained-bert


     |████████████████████████████████| 133kB 5.2MB/s 
     |████████████████████████████████| 102kB 8.9MB/s 
     |████████████████████████████████| 6.7MB 14.1MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
  Created wheel for boto3: filename=boto3-1.16.13-py2.py3-none-any.whl size=128453 sha256=7d006e87cfe9f9ae72c91fc6e410c5ddf75608cdb5acc9f1740ec06179e0cb37
  Stored in directory: /root/.cache/pip/wheels/bd/ca/cc/d99cff66806b87034af25f8fd8b0adb3e0151b17eea7891143
Successfully built boto3
ERROR: botocore 1.19.13 has requirement urllib3<1.26,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()


In [ ]:
import numpy as np
import torch
import os
import json


In [ ]:
!wget http://times.cs.uiuc.edu/~wang296/Data/LARA/TripAdvisor/TripAdvisorJson.tar.bz2


--2020-11-07 06:27:50--  http://times.cs.uiuc.edu/~wang296/Data/LARA/TripAdvisor/TripAdvisorJson.tar.bz2
Resolving times.cs.uiuc.edu (times.cs.uiuc.edu)... 192.17.58.170
Connecting to times.cs.uiuc.edu (times.cs.uiuc.edu)|192.17.58.170|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 455746769 (435M) [application/x-bzip2]
Saving to: ‘TripAdvisorJson.tar.bz2’

TripAdvisorJson.tar 100%[===================>] 434.63M  55.3MB/s    in 9.2s    

2020-11-07 06:28:00 (47.1 MB/s) - ‘TripAdvisorJson.tar.bz2’ saved [455746769/455746769]



In [ ]:
import shutil
shutil.unpack_archive("TripAdvisorJson.tar.bz2", "/content/")

In [ ]:
def replaceRating(rat):
    num = 0
    if(rat == "0.0" or rat == "0"):
        num= 0
    elif(rat == "1.0" or rat == "1"):
        num= 1
    elif(rat == "2.0" or rat == "2"):
        num= 2
    elif(rat == "3.0" or rat == "3"):
        num= 3
    elif(rat == "4.0" or rat == "4"):
        num= 4
    elif(rat == "5.0" or rat == "5"):
        num= 5
    return num

In [ ]:
# Set the maximum sequence length. 
MAX_LEN = 200

fileNames = os.listdir("json")
instances = [0,0,0,0,0,0]
maxInstances = 1935
contents= []
ratings = []
for fileName in fileNames:
    with open(("json/"+fileName)) as file:
        fileData = json.load(file)
        for review in fileData["Reviews"]:
            if ('Content' in review and len(review['Content']) != 0 and 'Overall' in review["Ratings"] and len(review["Ratings"]['Overall']) != 0):
                content = review["Content"]
                rating = review["Ratings"]["Overall"]
                rating = replaceRating(rating)
                
                if instances[rating] < maxInstances:
                    ratings.append(rating)
                    instances[rating] += 1

                    templength= len(content.split() )
                    if templength < MAX_LEN:
                        contents.append(content)
                    else:
                        temp= ""
                        convert= content.split()
                        for j in range(0, MAX_LEN):
                            temp= temp+ " "+convert[j]
                        contents.append(temp)    
                    

    if (instances[0] + instances[1] + instances[2]+instances[3]+instances[4]+ instances[5]) >= (maxInstances*6):
        break

    
print(len(contents))
print(len(ratings))
print(instances)

11610
11610
[1935, 1935, 1935, 1935, 1935, 1935]


In [ ]:
data_X = contents
data_Y= ratings


print(len(data_X))
print(len(data_Y))

11610
11610


In [ ]:
def extract_embed(data):
    embedings=np.zeros((len(data),768))
    cnt=0
    for i in data:
        # print(len(i.split()))
        
        # Add the special tokens.
        marked_text = "[CLS] " +i+" [SEP]"

        # Split the sentence into tokens.
        tokenized_text = tokenizer.tokenize(marked_text)
        #print (len(tokenized_text)
        
        # Map the token strings to their vocabulary indeces.
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        #print(indexed_tokens)
        # Mark each of the 22 tokens as belonging to sentence "1".
        segments_ids = [1] * len(tokenized_text)
        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        
        # Predict hidden states features for each layer
        with torch.no_grad():
            encoded_layers, _ = model(tokens_tensor, segments_tensors)
        # Concatenate the tensors for all layers. We use `stack` here to
        # create a new dimension in the tensor.
        token_embeddings = torch.stack(encoded_layers, dim=0)
        # Remove dimension 1, the "batches".
        token_embeddings = torch.squeeze(token_embeddings, dim=1)

        # Swap dimensions 0 and 1.
        token_embeddings = token_embeddings.permute(1,0,2)

        # `token_vecs` is a tensor with shape [22 x 768]
        token_vecs = encoded_layers[11][0]

        # Calculate the average of all 22 token vectors.
        sentence_embedding = torch.mean(token_vecs, dim=0)
        
        embedings[cnt]=sentence_embedding
        cnt+=1
        
    return embedings


In [ ]:
X= extract_embed( data_X )

In [ ]:
print( X.shape)
print( type(X))
Y= np.array(data_Y)
print( Y.shape)
print( type(Y))

(11610, 768)
<class 'numpy.ndarray'>
(11610,)
<class 'numpy.ndarray'>


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.15)



In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

classifier = SVC(kernel='linear')
# classifier = LogisticRegression(max_iter= 1000)
classifier.fit(X_train, y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred = classifier.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[297   0   0   0   0   1]
 [  0 187  70  17   3   3]
 [  1  84 125  60  10   2]
 [  2  17  64 120  52  17]
 [  0   5  12  69 153  65]
 [  3   3   4  11  89 196]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       298
           1       0.63      0.67      0.65       280
           2       0.45      0.44      0.45       282
           3       0.43      0.44      0.44       272
           4       0.50      0.50      0.50       304
           5       0.69      0.64      0.66       306

    accuracy                           0.62      1742
   macro avg       0.61      0.62      0.61      1742
weighted avg       0.62      0.62      0.62      1742

